<h3> CS3920/CS5920 Machine Learning Coursework</h3>

<h4> Assignment 1</h4>
    
This Assignment contains implemention of the following :<br>
- 1-Nearest Neighbor Classification Algorithm<br>
- Conformal Predictor 
    
This datasets are as follows:    
            -Iris Dataset<br>
            -Ionosphere Dataset<br>

<h4> Table of Contents </h4>

1. Exploring Datasets<br>
2. 1-NN Implementation<br>
3. Conformal Predictor

<h2>1. Exploring Datasets </h2>
<h3>1.1. Import Iris Dataset</h3>

train_test_split() from Scikit Learn Package, splits the data into test and train sets.<br>This is done with a 75-25 ratio for train and test respectively.<br>
random_state argument in test_train_split sets the seed to a specific state which helps to generate identical splits every time the function train_test_split() runs in order to have uniformity in every execution of the code 

In [1]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
from sklearn.model_selection import train_test_split
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(iris['data'],
iris['target'], random_state=1403)

In [2]:
print("Dimensions of train set:",X_train_iris.shape)
print("Dimensions of test set:",y_train_iris.shape)
print(type(iris))
print("Features:" ,iris.feature_names)
print("Target:", iris.target_names)

Dimensions of train set: (112, 4)
Dimensions of test set: (112,)
<class 'sklearn.utils.Bunch'>
Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target: ['setosa' 'versicolor' 'virginica']


<h3>1.2 Import Ionosphere Dataset</h3>

Function train_test_split() from Scikit Learn Package, splits the data into test and train sets.<br>This is done with a 75-25 ratio for train and test respectively.<br>
random_state argument in test_train_split sets the seed to a specific state which helps to generate identical splits every time the function train_test_split() runs in order to uniformity in every execution of the code 

Note: Path for dataset file needs to be changed. A local dummy path has been provided 

In [3]:
X = np.genfromtxt("E:\RHUL\Machine Learning\LABS\ionosphere.txt", delimiter=",", usecols = np.arange(34))
y = np.genfromtxt("E:\RHUL\Machine Learning\LABS\ionosphere.txt", delimiter=",",dtype = int, usecols=34)
X_train_ion, X_test_ion, y_train_ion, y_test_ion = train_test_split(X,
y, random_state=1403)

In [4]:
print("Dimensions of train set:",X_train_ion.shape)
print("Dimensions of test set:",y_train_ion.shape)
print("Type of data:" ,type(X))
from scipy import stats
print("Null value in train set:", np.isnan(X).all())
print("Null value in test set:", np.isnan(y).all())

Dimensions of train set: (263, 34)
Dimensions of test set: (263,)
Type of data: <class 'numpy.ndarray'>
Null value in train set: False
Null value in test set: False


<h3>2. 1-NN Implementation

<h4>2.1 Function for Distance calculation</h4>
    
Here a function has been defined for calculating distance between 2-arrays. The input can be in form of list/tuple/set. 
    

In [5]:
def dist(data_point_a,data_point_b):
    """
    function description

    Parameters
    ----------
    data_point_a : Iterable list/tuple
        This is a multi-dimensional point containing features of the data set. It can be train set/test set or any other
    data_point_b : Iterable list/tuple
        This is a multi-dimensional point containing features of the data set. It can be train set/test set or any other

    Returns
    -------
    dist_sq : float
    
    """
    dist_sq = 0    
    for i in range(len(data_point_b)):
        dist_sq = dist_sq + (data_point_a[i]-data_point_b[i])**2     
    return dist_sq**0.5

<h4>2.2 1-NN Function</h4>

This function is for calculating the test error rate for 1 Nearest Neighbor. It can be used on any data set. 

In [6]:
def knn(X_train,X_test,y_train,y_test):
    """
    This function is used to calculate the 1-NN. It calculates the minimum distance using the helper function dist().
    
    Parameters
    ----------
    X_train : numpy array
        Train set of features 
    X_test : numpy array
        Test set of features
    y_train : numpy array
        Train set of labels
    y_test : numpy array
        Test set of labels

    Returns
    -------
    TYPE : test error rate , count false values
        
    """
    y_pred = []
    count = 0
    #iterate through test sample and train sample for closest neighbour
    for test_sample in X_test:
        min_dist = np.inf
        for train_sample in range(len(X_train)):
            current_dist = dist(list(test_sample),list(X_train[train_sample]))                      
            if current_dist < min_dist :                
                min_dist = current_dist
                result = train_sample               
        y_pred.append(y_train[result])
    #count false predictions
    for label in range(len(y_pred)):
        if y_pred[label] !=y_test[label]:
            count = count + 1       
    return np.mean(y_pred!=y_test),count         

<h4>2.3 1-NN for Ionosphere</h4>
Function call for 1-NN for Ionosphere Dataset

In [7]:
test_error_rate_ion, mismatch_count_ion = knn( X_train_ion,X_test_ion, y_train_ion, y_test_ion)
print("Test error rate :",test_error_rate_ion)
print("Accuracy:", (1-test_error_rate_ion)*100)
print("Number of errors:",mismatch_count_ion)

Test error rate : 0.10227272727272728
Accuracy: 89.77272727272727
Number of errors: 9


<h4>2.4 1-NN for IRIS</h4>
Function call for 1-NN for Iris Dataset

In [8]:
test_error_rate_iris,mismatch_count_iris = knn(X_train_iris,X_test_iris, y_train_iris, y_test_iris)
print("Test error rate :",test_error_rate_iris)
print("Accuracy:", (1-test_error_rate_iris)*100)
print("Number of errors:",mismatch_count_iris)

Test error rate : 0.02631578947368421
Accuracy: 97.36842105263158
Number of errors: 1


<h3> 3. Conformal Prediction</h3>

In [9]:
def conformal_pred(X_train,X_test,y_train,y_test):   
    """
    This function implements the conformal predictor for dataset passed in the arguments.
    Following are the steps involved :
    1) Assume label for test set to be one among the unique train label set values
    2) Create extended training dataset: Add the assumed test set to the training set.
    3) For each element in the extended training dataset, calculate minimum same class distance , 
    minimum different class distance and update these values as you iterate through the set to find minimum
    4) Calculate the conformity score using c = minimum different class distance /minimum same class distance and store them
    5) Rank the above score by sorting and calculate the p values for each assumed label for each test label, create matrix
    6) From m*n matrix (m = number of test_label , n = number of unique labels), compare true labels from initial test data 
    and find the average false p-values 
    
    Parameters
    ----------
    X_train : numpy array
        Train set of features 
    X_test : numpy array
        Test set of features
    y_train : numpy array
        Train set of labels
    y_test : numpy array
        Test set of labels

    Returns
    -------
    TYPE : float, List
    Average false p value , matrix for p-values
        
    """
    p_value_false = []
    p_avg_false = []
    #loop to iterate over the test samples
    for test_sample in range(len(X_test)):  
        p_value_false.append([])
        rank = {}    
        #Loop to iterate over the number of unique labels in the data
        for i in range(len(set(y_test))):        
            conformity_score = []
            X_combined = np.append(X_train,[X_test[test_sample]],axis=0)
            y_combined = np.append(y_train,[np.unique(y_test)[i]],axis=0) 
            #Loop to iterate over extended train-set
            for train_sample in range(len(X_combined)):
                min_same_dist = np.inf
                min_diff_dist = np.inf
                #Loop to compare each element of extended train set and calculate min same/diff distance 
                for dist_element in range(len(X_combined)):                
                    if dist_element != train_sample:                   
                        if y_combined[dist_element] == y_combined[train_sample] :  
                            current_same_dist = dist(list(X_combined[dist_element]),list(X_combined[train_sample]))
                            if current_same_dist < min_same_dist:
                                min_same_dist = current_same_dist     
                        else:
                            current_diff_dist = dist(list(X_combined[dist_element]),list(X_combined[train_sample]))
                            if current_diff_dist < min_diff_dist:
                                min_diff_dist = current_diff_dist
                #calculation of confirmity score for each sample in the extended train set and saving it in a list                
                conformity_score.append({'index': train_sample, 'distance': min_diff_dist/min_same_dist})
            #sort the final conformity score to find out the ranks
            conformity_score = sorted(conformity_score,key = lambda i: i['distance'])
            #Find all p values and store in a matrix (list)
            p_val = (conformity_score.index(next(item for item in conformity_score if item["index"]==(len(X_combined))-1))+1)/len(X_combined)
            #store p values
            p_value_false[test_sample].append(p_val)

    #Average false p_value calculation         
    for test_label in range(len(y_test)):
        for i in range(len(set(y_test))):
                if np.unique(y_test)[i] != y_test[test_label]:
                    p_avg_false.append(p_value_false[test_label][i])                                          
    p_avg_false_values = sum(p_avg_false)/len(p_avg_false) 
    return p_avg_false_values, p_value_false   

<h4>3.1 Conformal Predictor for IRIS data set

In [10]:
avg_value, p_val_matrix = conformal_pred(X_train_iris,X_test_iris,y_train_iris,y_test_iris)
print("The average false p_values ", avg_value)
print("p_value matrix:")
print(p_val_matrix)

The average false p_values  0.010013972985561259
p_value matrix:
[[0.008849557522123894, 0.008849557522123894, 0.5486725663716814], [0.6637168141592921, 0.008849557522123894, 0.008849557522123894], [0.831858407079646, 0.008849557522123894, 0.008849557522123894], [0.672566371681416, 0.008849557522123894, 0.008849557522123894], [0.008849557522123894, 0.008849557522123894, 0.13274336283185842], [0.008849557522123894, 0.008849557522123894, 0.4336283185840708], [0.008849557522123894, 0.39823008849557523, 0.008849557522123894], [0.008849557522123894, 0.008849557522123894, 0.24778761061946902], [0.008849557522123894, 0.008849557522123894, 0.3805309734513274], [0.008849557522123894, 0.008849557522123894, 0.5132743362831859], [0.008849557522123894, 0.008849557522123894, 0.30973451327433627], [0.008849557522123894, 0.008849557522123894, 0.4336283185840708], [0.008849557522123894, 0.008849557522123894, 0.24778761061946902], [0.831858407079646, 0.008849557522123894, 0.008849557522123894], [0.00884

<h4>3.2 Conformal Predictor for Ionosphere dataset</h4>
Note : Ionosphere data is heavy for the conformal predictor for random state 1403 and takes more time than iris dataset
    

In [11]:
avg_value_ion, p_val_matrix_ion = conformal_pred(X_train_ion,X_test_ion,y_train_ion,y_test_ion)
print("The average false p_value :", avg_value_ion)
print("p_value matrix:")
print(p_val_matrix_ion)

The average false p_value : 0.05259986225895323
p_value matrix:
[[0.003787878787878788, 0.9128787878787878], [0.003787878787878788, 0.9128787878787878], [0.553030303030303, 0.011363636363636364], [0.003787878787878788, 0.7348484848484849], [0.4696969696969697, 0.015151515151515152], [0.3484848484848485, 0.06439393939393939], [0.003787878787878788, 0.7651515151515151], [0.003787878787878788, 0.9772727272727273], [0.007575757575757576, 0.6401515151515151], [0.2803030303030303, 0.0946969696969697], [0.015151515151515152, 0.4583333333333333], [0.003787878787878788, 0.7727272727272727], [0.2878787878787879, 0.09090909090909091], [0.003787878787878788, 0.8977272727272727], [0.003787878787878788, 0.7651515151515151], [0.125, 0.1590909090909091], [0.20454545454545456, 0.10984848484848485], [0.3333333333333333, 0.06439393939393939], [0.011363636363636364, 0.48863636363636365], [0.011363636363636364, 0.6287878787878788], [0.003787878787878788, 0.9772727272727273], [0.125, 0.16287878787878787], [